# Preprocessing package

The ``preprocessing`` package provides a number of functions for preprocessing data.
These functions include, categorical and numerical features encoders, classes that
can drop unnecessary columns automatically, such as, removing duplicate columns or
features with constant values (same value for every row of a given column).
Additionally, ``preprocessing`` also includes utilitarian functions. These utility
objects, allows you to perform useful operations including, reducing the memory
footprint of large datasets, impute missing values, and more.

## Objective

The goal of this notebook is to show some of the capabilities of the
``preprocessing`` package. Below you will find some examples on how this package can
 be used to accelerate the data preparation step.

In [3]:
import pandas as pd

from preprocessing.encoders.cat_encoders import LabelEncoder, SelectCategorical
from preprocessing.encoders.num_encoders import DistTransformer, SelectNumerical
from preprocessing.pipeline import Pipeline
from preprocessing.selector import (
    ConstantFeatureEliminator,
    DuplicatedFeatureEliminator,
)
from preprocessing.utils import compress_df

## Reading the data to be used as example

The input data to be used as example, is stored inside the ``neo_analytics/data/examples``
subdirectory. It consists of the training data being used to train a predictive model,
 that tries to identify customers that are most likely to default their credit card
 payments on the next month (relative to when the data was generated). The file
 consists of a CSV file, that has a mix of categorical and numerical features.
 Additionally, the target variable is also stored inside this input data. The
 target column name is ``"target"``.

In [4]:
df = pd.read_csv(r'../data/examples/train.csv')
df.head(5)


ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1      20000    2          2         1   24      2      2     -1     -1   
1   3      90000    2          2         2   34      0      0      0      0   
2   4      50000    2          2         1   37      0      0      0      0   
3   6      50000    1          1         2   37      0      0      0      0   
4   7     500000    1          1         2   29      0      0      0      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...          0          0          0         0       689         0   
1  ...      14331      14948      15549      1518      1500      1000   
2  ...      28314      28959      29547      2000      2019      1200   
3  ...      19394      19619      20024      2500      1815       657   
4  ...     542653     483003     473944     55000     40000     38000   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  target  
0         0         0         0       1  
1      1000      1000      5000       0  
2      1100      1069      1000       0  
3      1000      1000       800       0  
4     20239     13750     13770       0  

[5 rows x 25 columns]

## Preparing the data

The first thing that might be useful to do is to drop some of the columns that we know
beforehand that should not be included in the training features, that get fed into the
model. In the input file being used as example, we should drop the column "ID", since it
represents the unique identification number of each of the customers, and does not contain
any useful information that can help the model to predict the target variable.

Because our input data already consists of only numerical columns, the code below also
converts the "SEX" column back to strings (with 'M' and 'F' labels). This in turn will
allows us to test the categorical features later on. Finally, a column with the name
'CONSTANT_FEAT' is also being created and populated with a constant values, to test
the features exclusion functionality.

In [36]:
cols_to_drop = ['ID']
df = df.drop(columns=cols_to_drop, errors='ignore')
df['SEX'] = df['SEX'].astype(int).map(lambda x: 'M' if x == 1 else 'F')
df['CONSTANT_FEAT'] = 'FOO'
df.head(5)

LIMIT_BAL SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  PAY_5  \
0      20000   F          2         1   24      2      2     -1     -1     -2   
1      90000   F          2         2   34      0      0      0      0      0   
2      50000   F          2         1   37      0      0      0      0      0   
3      50000   M          1         2   37      0      0      0      0      0   
4     500000   M          1         2   29      0      0      0      0      0   

   ...  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  PAY_AMT4  \
0  ...          0          0         0       689         0         0   
1  ...      14948      15549      1518      1500      1000      1000   
2  ...      28959      29547      2000      2019      1200      1100   
3  ...      19619      20024      2500      1815       657      1000   
4  ...     483003     473944     55000     40000     38000     20239   

   PAY_AMT5  PAY_AMT6  target  CONSTANT_FEAT  
0         0         0       1            FOO  
1      1000      5000       0            FOO  
2      1069      1000       0            FOO  
3      1000       800       0            FOO  
4     13750     13770       0            FOO  

[5 rows x 25 columns]

## Numerical Encoders

The following pipeline will find all numerical features in our dataset, and normalize their
values to be between 0 and 1, using a 'min-max' scaler strategy.

In [37]:
num_feats_encoder = Pipeline(
    [
        ConstantFeatureEliminator(),
        DuplicatedFeatureEliminator(),
        SelectNumerical(),
        DistTransformer(transform='min-max')
    ]
)


In [38]:
num_df = num_feats_encoder.fit_transform(df)
num_df.head(5)


LIMIT_BAL  EDUCATION  MARRIAGE       AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   0.010101   0.333333  0.333333  0.051724    0.4    0.4    0.1    0.1   
1   0.080808   0.333333  0.666667  0.224138    0.2    0.2    0.2    0.2   
2   0.040404   0.333333  0.333333  0.275862    0.2    0.2    0.2    0.2   
3   0.040404   0.166667  0.666667  0.275862    0.2    0.2    0.2    0.2   
4   0.494949   0.166667  0.666667  0.137931    0.2    0.2    0.2    0.2   

   PAY_5  PAY_6  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  \
0    0.0    0.0  ...   0.083598   0.080648   0.260979  0.000000  0.000409   
1    0.2    0.2  ...   0.098328   0.095470   0.272928  0.003006  0.000891   
2    0.2    0.2  ...   0.112700   0.109363   0.283685  0.003960  0.001199   
3    0.2    0.2  ...   0.103532   0.100102   0.276367  0.004950  0.001078   
4    0.2    0.2  ...   0.641355   0.559578   0.625196  0.108911  0.023749   

   PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  target  
0  0.000000  0.000000  0.000000  0.000000     1.0  
1  0.001116  0.001891  0.002345  0.009458     0.0  
2  0.001339  0.002080  0.002506  0.001892     0.0  
3  0.000733  0.001891  0.002345  0.001513     0.0  
4  0.042409  0.038266  0.032237  0.026047     0.0  

[5 rows x 23 columns]

## Categorical Encoders

The following pipeline will find all categorical features in our dataset, and convert them
to numerical representations of each label.

In [39]:
cat_feats_encoder = Pipeline(
    [
        ConstantFeatureEliminator(),
        DuplicatedFeatureEliminator(),
        SelectCategorical(),
        LabelEncoder(),
    ]
)
cat_df = cat_feats_encoder.fit_transform(df)
cat_df.head(5)


SEX  SEX_le
0   F       0
1   F       0
2   F       0
3   M       1
4   M       1

### Note about the encoders

As you can see, it is quite easy to combine multiple encoders in sequence. These
classes are designed to reduce the need of having to define various variables
during instantiation.


## Combining the numerical and categorical dataframes

In [40]:
processed_df = pd.concat([num_df, cat_df], axis=1)
processed_df.head(5)


LIMIT_BAL  EDUCATION  MARRIAGE       AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   0.010101   0.333333  0.333333  0.051724    0.4    0.4    0.1    0.1   
1   0.080808   0.333333  0.666667  0.224138    0.2    0.2    0.2    0.2   
2   0.040404   0.333333  0.333333  0.275862    0.2    0.2    0.2    0.2   
3   0.040404   0.166667  0.666667  0.275862    0.2    0.2    0.2    0.2   
4   0.494949   0.166667  0.666667  0.137931    0.2    0.2    0.2    0.2   

   PAY_5  PAY_6  ...  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  PAY_AMT4  \
0    0.0    0.0  ...   0.260979  0.000000  0.000409  0.000000  0.000000   
1    0.2    0.2  ...   0.272928  0.003006  0.000891  0.001116  0.001891   
2    0.2    0.2  ...   0.283685  0.003960  0.001199  0.001339  0.002080   
3    0.2    0.2  ...   0.276367  0.004950  0.001078  0.000733  0.001891   
4    0.2    0.2  ...   0.625196  0.108911  0.023749  0.042409  0.038266   

   PAY_AMT5  PAY_AMT6  target  SEX  SEX_le  
0  0.000000  0.000000     1.0    F       0  
1  0.002345  0.009458     0.0    F       0  
2  0.002506  0.001892     0.0    F       0  
3  0.002345  0.001513     0.0    M       1  
4  0.032237  0.026047     0.0    M       1  

[5 rows x 25 columns]

In [41]:
processed_df.to_csv(r'../data/examples/train_processed.csv')


### Extra: Compressing the dataframe

For large datasets, the function ``compress_df`` can be used to reduce the memory footprint
of the dataframe, by converting its columns to the smallest possible types.

In [43]:
optimized_processed_df = compress_df(processed_df, verbose=True)


Memory reduced from 4.58 MB to 2.31 MB
